In [ ]:
!nvidia-smi

Wed May  7 09:06:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ymmjg1yd
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ymmjg1yd
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-tn8dbtxa/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpgp_boz1k".


In [ ]:
%%cuda
#include <iostream>

int main(){

    std::cout << "Hello World\n";

    return 0;
}

Hello World



In [ ]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>

__global__ void matmul(int* A, int* B, int* C, int N) {
    int Row = blockIdx.y*blockDim.y+threadIdx.y;
    int Col = blockIdx.x*blockDim.x+threadIdx.x;
    if (Row < N && Col < N) {
        int Pvalue = 0;
        for (int k = 0; k < N; k++) {
            Pvalue += A[Row*N+k] * B[k*N+Col];
        }
        C[Row*N+Col] = Pvalue;
    }
}

int main() {
    int N = 512;
    int size = N * N * sizeof(int);
    int* A, * B, * C;
    int* dev_A, * dev_B, * dev_C;
    cudaMallocHost(&A, size);
    cudaMallocHost(&B, size);
    cudaMallocHost(&C, size);
    cudaMalloc(&dev_A, size);
    cudaMalloc(&dev_B, size);
    cudaMalloc(&dev_C, size);

    // Initialize matrices A and B
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            A[i*N+j] = i*N+j;
            B[i*N+j] = j*N+i;
        }
    }

    cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(16, 16);
    dim3 dimGrid(N/dimBlock.x, N/dimBlock.y);

    matmul<<<dimGrid, dimBlock>>>(dev_A, dev_B, dev_C, N);

    cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    // Print the result
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            std::cout << C[i*N+j] << " ";
        }
        std::cout << std::endl;
    }

    // Free memory
    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C);

    return 0;
}

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 



In [ ]:
%%cuda -arch=sm_75 --cudart=shared
#include <cuda_runtime.h>
#include <iostream>

__global__ void matmul(int* A, int* B, int* C, int N) {
    int Row = blockIdx.y * blockDim.y + threadIdx.y;
    int Col = blockIdx.x * blockDim.x + threadIdx.x;

    if (Row < N && Col < N) {
        int Pvalue = 0;
        for (int k = 0; k < N; k++) {
            Pvalue += A[Row * N + k] * B[k * N + Col];
        }
        C[Row * N + Col] = Pvalue;
    }
}

int main() {
    int N = 512;
    int size = N * N * sizeof(int);
    int *A, *B, *C;
    int *dev_A, *dev_B, *dev_C;

    // Allocate memory
    cudaMallocHost(&A, size);
    cudaMallocHost(&B, size);
    cudaMallocHost(&C, size);
    cudaMalloc(&dev_A, size);
    cudaMalloc(&dev_B, size);
    cudaMalloc(&dev_C, size);

    // Initialize A and B
    for (int i = 0; i < N; ++i)
        for (int j = 0; j < N; ++j) {
            A[i * N + j] = i * N + j;
            B[i * N + j] = j * N + i;
        }

    cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(16, 16);
    dim3 dimGrid((N + dimBlock.x - 1) / dimBlock.x,
                 (N + dimBlock.y - 1) / dimBlock.y);

    // Launch kernel
    matmul<<<dimGrid, dimBlock>>>(dev_A, dev_B, dev_C, N);

    // Check for errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        std::cerr << "CUDA error: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    // Synchronize
    cudaDeviceSynchronize();

    cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    // Print top-left 10x10 corner
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            std::cout << C[i * N + j] << " ";
        }
        std::cout << std::endl;
    }

    // Free memory
    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C);

    return 0;
}


usage: colab_kernel_launcher.py [-h] [-t] [-p] [-l PROFILER]
                                [-a PROFILER_ARGS] [-c COMPILER_ARGS]

%%cuda magic that compiles and runs CUDA C++ code in this cell. See
https://nvcc4jupyter.readthedocs.io/en/latest/magics.html#cuda for usage
details.

options:
  -h, --help            show this help message and exit
  -t, --timeit
  -p, --profile
  -l PROFILER, --profiler PROFILER
  -a PROFILER_ARGS, --profiler-args PROFILER_ARGS
  -c COMPILER_ARGS, --compiler-args COMPILER_ARGS


usage: colab_kernel_launcher.py [-h] [-t] [-p] [-l PROFILER]
                                [-a PROFILER_ARGS] [-c COMPILER_ARGS]
colab_kernel_launcher.py: error: unrecognized arguments: --cudart=shared


In [ ]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>

__global__ void matmul(int* A, int* B, int* C, int N) {
    int Row = blockIdx.y*blockDim.y+threadIdx.y;
    int Col = blockIdx.x*blockDim.x+threadIdx.x;
    if (Row < N && Col < N) {
        int Pvalue = 0;
        for (int k = 0; k < N; k++) {
            Pvalue += A[Row*N+k] * B[k*N+Col];
        }
        C[Row*N+Col] = Pvalue;
    }
}

int main() {
    int N = 512;
    int size = N * N * sizeof(int);
    int* A, * B, * C;
    int* dev_A, * dev_B, * dev_C;
    cudaMallocHost(&A, size);
    cudaMallocHost(&B, size);
    cudaMallocHost(&C, size);
    cudaMalloc(&dev_A, size);
    cudaMalloc(&dev_B, size);
    cudaMalloc(&dev_C, size);

    // Initialize matrices A and B
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            A[i*N+j] = i*N+j;
            B[i*N+j] = j*N+i;
        }
    }

    cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(16, 16);
    dim3 dimGrid(N/dimBlock.x, N/dimBlock.y);

    matmul<<<dimGrid, dimBlock>>>(dev_A, dev_B, dev_C, N);

    cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    // Print the result
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            std::cout << C[i*N+j] << " ";
        }
        std::cout << std::endl;
    }

    // Free memory
    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C);

    return 0;
}

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 



In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>
#include <stdlib.h>
#include <time.h>

void initialization(int *a, int *b, int length);

__global__ void add(int *a, int *b, int *c, int N) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < N) {
        c[id] = a[id] + b[id];
    }
}

int main() {
    int *h_a, *h_b, *h_c;
    int *dev_a, *dev_b, *dev_c;
    int max = 1 << 29;  // Maximum size (2^29 elements)

    clock_t start_t, end_t;
    double walltime, th;
    int size = 0;

    // Timing using CUDA events
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    for (int j = 256; j <= max; j *= 2) {
        size = j * sizeof(int);
        // Allocate memory for host variables
        h_a = (int *)malloc(size);
        h_b = (int *)malloc(size);
        h_c = (int *)malloc(size);

        // Initialize host variables
        initialization(h_a, h_b, j);

        // Allocate memory for device variables
        cudaMalloc((void **)&dev_a, size);
        cudaMalloc((void **)&dev_b, size);
        cudaMalloc((void **)&dev_c, size);

        // Copy host to device
        cudaMemcpy(dev_a, h_a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(dev_b, h_b, size, cudaMemcpyHostToDevice);

        // Start timing
        cudaEventRecord(start);

        // Launch kernel with dynamic number of blocks and threads per block
        int blockSize = 256;
        int numBlocks = (j + blockSize - 1) / blockSize;  // Ensure full coverage of all indices
        add<<<numBlocks, blockSize>>>(dev_a, dev_b, dev_c, j);

        // Stop timing
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        // Calculate the elapsed time
        float elapsedTime;
        cudaEventElapsedTime(&elapsedTime, start, stop);
        printf("Time taken for vector size %d: %f ms\n", j, elapsedTime);

        // Copy result back to host
        cudaMemcpy(h_c, dev_c, size, cudaMemcpyDeviceToHost);

        // Optionally, print the first 10 elements of the result to verify correctness
        for (int i = 0; i < 10; i++) {
            printf("%d + %d = %d\n", h_a[i], h_b[i], h_c[i]);
        }

        // Free memory
        free(h_a);
        free(h_b);
        free(h_c);
        cudaFree(dev_a);
        cudaFree(dev_b);
        cudaFree(dev_c);
    }

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

void initialization(int *a, int *b, int length) {
    for (int i = 0; i < length; i++) {
        a[i] = 1;
        b[i] = i + 1;
    }
}


In [ ]:
%%cuda
#include <iostream>
#include <cstdlib>
#include <cuda_runtime.h>

using namespace std;

// Kernel function to add two vectors
__global__ void vectorAdd(int *a, int *b, int *result, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < n) {
        result[tid] = a[tid] + b[tid];
    }
}

int main() {
    // Define vector size
    int n = 1 << 24; // 16 million elements (2^24)
    int *a, *b, *c; // Host pointers
    int *d_a, *d_b, *d_c; // Device pointers

    // Allocate memory for vectors on the host
    a = new int[n];
    b = new int[n];
    c = new int[n];

    // Allocate memory for vectors on the device
    cudaMalloc((void**)&d_a, n * sizeof(int));
    cudaMalloc((void**)&d_b, n * sizeof(int));
    cudaMalloc((void**)&d_c, n * sizeof(int));

    // Initialize vectors a and b
    for (int i = 0; i < n; i++) {
        a[i] = 1;  // You can randomize the values if needed
        b[i] = 2;
    }

    // Copy input vectors from host to device
    cudaMemcpy(d_a, a, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, n * sizeof(int), cudaMemcpyHostToDevice);

    // Define block and grid size
    int threadsPerBlock = 1024;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

    // Launch the kernel
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    // Check for errors during kernel launch
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        cout << "CUDA error: " << cudaGetErrorString(err) << endl;
        return -1;
    }

    // Copy result from device to host
    cudaMemcpy(c, d_c, n * sizeof(int), cudaMemcpyDeviceToHost);

    // Verify the result
    int error = 0;
    for (int i = 0; i < n; i++) {
        if (c[i] != 3) {  // Expected result for 1 + 2
            error++;
        }
    }

    // Output result
    if (error == 0) {
        cout << "Vector addition successful!" << endl;
    } else {
        cout << "There were " << error << " errors in the vector addition." << endl;
    }

    // Free memory
    delete[] a;
    delete[] b;
    delete[] c;
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


CUDA error: the provided PTX was compiled with an unsupported toolchain.



In [ ]:
%%cuda
#include <stdio.h>

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )

static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}



const int N = 2048;

// CUDA Kernel for Vector Addition
__global__ void Vector_Addition ( const int *dev_a , const int *dev_b , int *dev_c)
{
      //Get the id of thread within a block
      unsigned int tid = blockIdx.x * blockDim.x + threadIdx.x ;


      while ( tid < N ) // check the boundry condition for the threads
      {
            dev_c [tid] = dev_a[tid] + dev_b[tid] ;

            tid+= blockDim.x * gridDim.x ;
      }
}

int main (void)
{

      //Host array
      int Host_a[N], Host_b[N], Host_c[N];

      //Device array
      int *dev_a , *dev_b, *dev_c ;

      //Allocate the memory on the GPU
      HANDLE_ERROR ( cudaMalloc((void **)&dev_a , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_b , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_c , N*sizeof(int) ) );

      //fill the Host array with random elements on the CPU
      for ( int i = 0; i <N ; i++ )
      {
            Host_a[i] = -i ;
            Host_b[i] = i*i ;
      }

      //Copy Host array to Device array
      HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , N*sizeof(int) , cudaMemcpyHostToDevice));
      HANDLE_ERROR (cudaMemcpy (dev_b , Host_b , N*sizeof(int) , cudaMemcpyHostToDevice));

      //Make a call to GPU kernel
      Vector_Addition <<< (N+127)/128, 128  >>> (dev_a , dev_b , dev_c ) ;

      //Copy back to Host array from Device array
      HANDLE_ERROR (cudaMemcpy(Host_c , dev_c , N*sizeof(int) , cudaMemcpyDeviceToHost));

      //Display the result
      for ( int i = 0; i<N; i++ )
                  printf ("%d + %d = %d\n", Host_a[i] , Host_b[i] , Host_c[i] ) ;

      //Free the Device array memory
      cudaFree (dev_a) ;
      cudaFree (dev_b) ;
      cudaFree (dev_c) ;

      system("pause");
      return 0 ;

}

0 + 0 = 0
-1 + 1 = 0
-2 + 4 = 0
-3 + 9 = 0
-4 + 16 = 0
-5 + 25 = 0
-6 + 36 = 0
-7 + 49 = 0
-8 + 64 = 0
-9 + 81 = 0
-10 + 100 = 0
-11 + 121 = 0
-12 + 144 = 0
-13 + 169 = 0
-14 + 196 = 0
-15 + 225 = 0
-16 + 256 = 0
-17 + 289 = 0
-18 + 324 = 0
-19 + 361 = 0
-20 + 400 = 0
-21 + 441 = 0
-22 + 484 = 0
-23 + 529 = 0
-24 + 576 = 0
-25 + 625 = 0
-26 + 676 = 0
-27 + 729 = 0
-28 + 784 = 0
-29 + 841 = 0
-30 + 900 = 0
-31 + 961 = 0
-32 + 1024 = 0
-33 + 1089 = 0
-34 + 1156 = 0
-35 + 1225 = 0
-36 + 1296 = 0
-37 + 1369 = 0
-38 + 1444 = 0
-39 + 1521 = 0
-40 + 1600 = 0
-41 + 1681 = 0
-42 + 1764 = 0
-43 + 1849 = 0
-44 + 1936 = 0
-45 + 2025 = 0
-46 + 2116 = 0
-47 + 2209 = 0
-48 + 2304 = 0
-49 + 2401 = 0
-50 + 2500 = 0
-51 + 2601 = 0
-52 + 2704 = 0
-53 + 2809 = 0
-54 + 2916 = 0
-55 + 3025 = 0
-56 + 3136 = 0
-57 + 3249 = 0
-58 + 3364 = 0
-59 + 3481 = 0
-60 + 3600 = 0
-61 + 3721 = 0
-62 + 3844 = 0
-63 + 3969 = 0
-64 + 4096 = 0
-65 + 4225 = 0
-66 + 4356 = 0
-67 + 4489 = 0
-68 + 4624 = 0
-69 + 4761 = 0
-70 + 4

In [ ]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>

__global__ void vector_add(int* A, int* B, int* C, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        C[idx] = A[idx] + B[idx];
    }
}

int main() {
    int N = 1 << 20; // Large vector size (1 million elements)
    int size = N * sizeof(int);

    int* A, * B, * C;
    int* dev_A, * dev_B, * dev_C;

    // Allocate host memory
    cudaError_t err = cudaMallocHost(&A, size);
    if (err != cudaSuccess) {
        std::cerr << "CUDA mallocHost failed for A: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    err = cudaMallocHost(&B, size);
    if (err != cudaSuccess) {
        std::cerr << "CUDA mallocHost failed for B: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    err = cudaMallocHost(&C, size);
    if (err != cudaSuccess) {
        std::cerr << "CUDA mallocHost failed for C: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    // Allocate device memory
    err = cudaMalloc(&dev_A, size);
    if (err != cudaSuccess) {
        std::cerr << "CUDA malloc failed for dev_A: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    err = cudaMalloc(&dev_B, size);
    if (err != cudaSuccess) {
        std::cerr << "CUDA malloc failed for dev_B: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    err = cudaMalloc(&dev_C, size);
    if (err != cudaSuccess) {
        std::cerr << "CUDA malloc failed for dev_C: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    // Initialize vectors A and B
    for (int i = 0; i < N; i++) {
        A[i] = i;
        B[i] = i;
    }

    // Copy data to device
    err = cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        std::cerr << "CUDA memcpy failed for dev_A: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    err = cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        std::cerr << "CUDA memcpy failed for dev_B: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    // Launch kernel with 256 threads per block and N / 256 blocks
    int blockSize = 256;
    int gridSize = (N + blockSize - 1) / blockSize;
    vector_add<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, N);

    // Check for kernel launch errors
    err = cudaGetLastError();
    if (err != cudaSuccess) {
        std::cerr << "CUDA error after kernel launch: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    // Copy result back to host
    err = cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
        std::cerr << "CUDA memcpy failed for dev_C: " << cudaGetErrorString(err) << std::endl;
        return -1;
    }

    // Output the first 10 elements of the result
    for (int i = 0; i < 10; i++) {
        std::cout << C[i] << " ";
    }
    std::cout << std::endl;

    // Free memory
    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C);

    return 0;
}


CUDA error after kernel launch: the provided PTX was compiled with an unsupported toolchain.



In [ ]:
%%cuda --arch=sm_61
#include <iostream>
__global__ void vector_add(int* A, int* B, int* C, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        C[idx] = A[idx] + B[idx];
    }
}
int main() {
    int N = 1 << 20; // Large vector size (1 million elements)
    int size = N * sizeof(int);
    int* A, * B, * C;
    int* dev_A, * dev_B, * dev_C;
    // Allocate host memory
    cudaMallocHost(&A, size);
    cudaMallocHost(&B, size);
    cudaMallocHost(&C, size);
    // Allocate device memory
    cudaMalloc(&dev_A, size);
    cudaMalloc(&dev_B, size);
    cudaMalloc(&dev_C, size);
    // Initialize vectors A and B
    for (int i = 0; i < N; i++) {
        A[i] = i;
        B[i] = i;
    }
    // Copy data to device
    cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);
    // Launch kernel with 256 threads per block and N / 256 blocks
    int blockSize = 256;
    int gridSize = (N + blockSize - 1) / blockSize;
    vector_add<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, N);
    // Copy result back to host
    cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);
    // Output the first 10 elements of the result
    for (int i = 0; i < 10; i++) {
        std::cout << C[i] << " ";
    }
    std::cout << std::endl;
    // Free memory
    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C);
    return 0;
}


usage: colab_kernel_launcher.py [-h] [-t] [-p] [-l PROFILER]
                                [-a PROFILER_ARGS] [-c COMPILER_ARGS]

%%cuda magic that compiles and runs CUDA C++ code in this cell. See
https://nvcc4jupyter.readthedocs.io/en/latest/magics.html#cuda for usage
details.

options:
  -h, --help            show this help message and exit
  -t, --timeit
  -p, --profile
  -l PROFILER, --profiler PROFILER
  -a PROFILER_ARGS, --profiler-args PROFILER_ARGS
  -c COMPILER_ARGS, --compiler-args COMPILER_ARGS


usage: colab_kernel_launcher.py [-h] [-t] [-p] [-l PROFILER]
                                [-a PROFILER_ARGS] [-c COMPILER_ARGS]
colab_kernel_launcher.py: error: unrecognized arguments: --arch=sm_61


In [ ]:
%%cuda
#include<stdio.h>
#include<cuda.h>

__global__ void arradd(int *x, int *y, int *z, int N) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < N) {  // Ensure the thread index is within bounds
        z[id] = x[id] + y[id];
    }
}

int main() {
    int N = 1 << 20;  // Size of the vectors (1 million elements)
    int size = N * sizeof(int);

    int *a, *b, *c;  // Host arrays
    int *d, *e, *f;  // Device arrays
    int i;

    // Allocate memory for the vectors on the host (CPU)
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize the vectors with some values (e.g., integers from 1 to N)
    for(i = 0; i < N; i++) {
        a[i] = i + 1;  // Vector A: 1, 2, 3, ..., N
        b[i] = i + 1;  // Vector B: 1, 2, 3, ..., N
    }

    // Allocate memory on the device (GPU)
    cudaMalloc((void**)&d, size);
    cudaMalloc((void**)&e, size);
    cudaMalloc((void**)&f, size);

    // Copy data from host to device
    cudaMemcpy(d, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, size, cudaMemcpyHostToDevice);

    // Set block size and grid size
    int blockSize = 256;  // Number of threads per block
    int gridSize = (N + blockSize - 1) / blockSize;  // Calculate number of blocks

    // Launch the kernel to add vectors
    arradd<<<gridSize, blockSize>>>(d, e, f, N);

    // Copy the result from device to host
    cudaMemcpy(c, f, size, cudaMemcpyDeviceToHost);

    // Print the first 10 elements of the result (for verification)
    printf("Sum of the first 10 elements of two large vectors:\n");
    for(i = 0; i < 10; i++) {
        printf("%d ", c[i]);
    }
    printf("\n");

    // Free allocated memory
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    free(a);
    free(b);
    free(c);

    return 0;
}


Sum of the first 10 elements of two large vectors:
0 0 0 0 0 0 0 0 0 0 



In [ ]:
%%cuda --compiler-args="-arch=sm_70"

#include<stdio.h>
#include<cuda.h>

__global__ void arradd(int *x, int *y, int *z, int N) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < N) {  // Ensure the thread index is within bounds
        z[id] = x[id] + y[id];
    }
}

int main() {
    int N = 1 << 20;  // Size of the vectors (1 million elements)
    int size = N * sizeof(int);

    int *a, *b, *c;  // Host arrays
    int *d, *e, *f;  // Device arrays
    int i;

    // Allocate memory for the vectors on the host (CPU)
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize the vectors with some values (e.g., integers from 1 to N)
    for(i = 0; i < N; i++) {
        a[i] = i + 1;  // Vector A: 1, 2, 3, ..., N
        b[i] = i + 1;  // Vector B: 1, 2, 3, ..., N
    }

    // Allocate memory on the device (GPU)
    cudaMalloc((void**)&d, size);
    cudaMalloc((void**)&e, size);
    cudaMalloc((void**)&f, size);

    // Copy data from host to device
    cudaMemcpy(d, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, size, cudaMemcpyHostToDevice);

    // Set block size and grid size
    int blockSize = 256;  // Number of threads per block
    int gridSize = (N + blockSize - 1) / blockSize;  // Calculate number of blocks

    // Launch the kernel to add vectors
    arradd<<<gridSize, blockSize>>>(d, e, f, N);

    // Check for any errors during kernel execution
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }

    // Ensure all threads have completed
    cudaDeviceSynchronize();

    // Copy the result from device to host
    cudaMemcpy(c, f, size, cudaMemcpyDeviceToHost);

    // Check for any errors during memory copy
    err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error during memory copy: %s\n", cudaGetErrorString(err));
    }

    // Print the first 10 elements of the result (for verification)
    printf("Sum of the first 10 elements of two large vectors:\n");
    for(i = 0; i < 10; i++) {
        printf("%d ", c[i]);
    }
    printf("\n");

    // Free allocated memory
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    free(a);
    free(b);
    free(c);

    return 0;
}


CUDA Error: the provided PTX was compiled with an unsupported toolchain.
Sum of the first 10 elements of two large vectors:
0 0 0 0 0 0 0 0 0 0 



In [ ]:
%%cuda --compiler-args="-arch=sm_70"
#include <stdio.h>

__global__ void hello_world() {
    printf("Hello from CUDA Kernel!\n");
}

int main() {
    hello_world<<<1, 1>>>();
    cudaDeviceSynchronize();  // Ensure kernel finishes execution
    return 0;
}


In [ ]:
%%cuda --compiler-args="-arch=sm_70"
#include <stdio.h>

__global__ void hello_world() {
    printf("Hello from CUDA Kernel!\n");
}

// Launch the kernel outside of main
hello_world<<<1, 1>>>();
cudaDeviceSynchronize();  // Wait for the kernel to finish


/tmp/tmpgp_boz1k/b54592df-d853-4d1a-9008-ecbe9bff904d/single_file.cu(8): error: this declaration has no storage class or type specifier
  hello_world<<<1, 1>>>();
  ^

/tmp/tmpgp_boz1k/b54592df-d853-4d1a-9008-ecbe9bff904d/single_file.cu(8): error: declaration is incompatible with "void hello_world()" (declared at line 3)
  hello_world<<<1, 1>>>();
  ^

/tmp/tmpgp_boz1k/b54592df-d853-4d1a-9008-ecbe9bff904d/single_file.cu(8): error: expected a ";"
  hello_world<<<1, 1>>>();
             ^

/tmp/tmpgp_boz1k/b54592df-d853-4d1a-9008-ecbe9bff904d/single_file.cu(9): error: explicit type is missing ("int" assumed)
  cudaDeviceSynchronize();
  ^

/tmp/tmpgp_boz1k/b54592df-d853-4d1a-9008-ecbe9bff904d/single_file.cu(9): error: cannot overload functions distinguished by return type alone
  cudaDeviceSynchronize();
  ^

5 errors detected in the compilation of "/tmp/tmpgp_boz1k/b54592df-d853-4d1a-9008-ecbe9bff904d/single_file.cu".



In [ ]:
%%cuda --compiler-args="-arch=sm_70"
#include <stdio.h>

__global__ void hello_world() {
    printf("Hello from CUDA Kernel!\n");
}

extern "C" void run() {
    hello_world<<<1, 1>>>();
    cudaDeviceSynchronize();
}



/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x1b): undefined reference to `main'
collect2: error: ld returned 1 exit status



In [ ]:
%%cuda --compiler-args="-arch=sm_70"
#include <stdio.h>

__global__ void hello_world() {
    printf("Hello from CUDA Kernel!\n");
}

int main() {
    hello_world<<<1, 1>>>();
    cudaDeviceSynchronize();  // Ensure kernel finishes before program ends
    return 0;
}


In [ ]:
%%cuda --compiler-args="-arch=sm_70"
#include <stdio.h>

__global__ void vector_add(const float *A, const float *B, float *C, int N) {
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    if (idx < N) {
        C[idx] = A[idx] + B[idx];
    }
}

int main() {
    const int N = 10;
    float h_A[N], h_B[N], h_C[N];
    float *d_A, *d_B, *d_C;

    // Initialize vectors
    for (int i = 0; i < N; i++) {
        h_A[i] = i;
        h_B[i] = i * 2;
    }

    cudaMalloc((void**)&d_A, N * sizeof(float));
    cudaMalloc((void**)&d_B, N * sizeof(float));
    cudaMalloc((void**)&d_C, N * sizeof(float));

    cudaMemcpy(d_A, h_A, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, N * sizeof(float), cudaMemcpyHostToDevice);

    vector_add<<<1, N>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_C, d_C, N * sizeof(float), cudaMemcpyDeviceToHost);

    printf("Result of vector addition:\n");
    for (int i = 0; i < N; i++) {
        printf("%0.1f + %0.1f = %0.1f\n", h_A[i], h_B[i], h_C[i]);
    }

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Result of vector addition:
0.0 + 0.0 = 0.0
1.0 + 2.0 = 0.0
2.0 + 4.0 = 0.0
3.0 + 6.0 = 0.0
4.0 + 8.0 = 0.0
5.0 + 10.0 = 0.0
6.0 + 12.0 = 0.0
7.0 + 14.0 = 0.0
8.0 + 16.0 = 0.0
9.0 + 18.0 = 0.0



In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>

#define row1 2 /* Number of rows of first matrix */
#define col1 3 /* Number of columns of first matrix */
#define row2 3 /* Number of rows of second matrix */
#define col2 2 /* Number of columns of second matrix */

__global__ void matproduct(int *l, int *m, int *n) {
    int x = blockIdx.x;
    int y = blockIdx.y;
    int k;

    // Ensuring we stay within bounds
    if (x < col2 && y < row1) {
        n[col2 * y + x] = 0;
        for (k = 0; k < col1; k++) {
            n[col2 * y + x] += l[col1 * y + k] * m[col2 * k + x];
        }
    }
}

int main() {
    int a[row1][col1];
    int b[row2][col2];
    int c[row1][col2];
    int *d, *e, *f;
    int i, j;

    // Input matrices
    printf("\nEnter elements of first matrix of size 2x3\n");
    for (i = 0; i < row1; i++) {
        for (j = 0; j < col1; j++) {
            scanf("%d", &a[i][j]);
        }
    }

    printf("\nEnter elements of second matrix of size 3x2\n");
    for (i = 0; i < row2; i++) {
        for (j = 0; j < col2; j++) {
            scanf("%d", &b[i][j]);
        }
    }

    // Memory allocation on the device
    cudaMalloc((void **)&d, row1 * col1 * sizeof(int));
    cudaMalloc((void **)&e, row2 * col2 * sizeof(int));
    cudaMalloc((void **)&f, row1 * col2 * sizeof(int));

    // Copy matrices to device memory
    cudaMemcpy(d, a, row1 * col1 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, row2 * col2 * sizeof(int), cudaMemcpyHostToDevice);

    // Define grid dimensions
    dim3 grid(col2, row1);

    // Launch kernel
    matproduct<<<grid, 1>>>(d, e, f);

    // Check for errors after kernel launch
    cudaError_t errSync = cudaGetLastError();
    cudaError_t errAsync = cudaDeviceSynchronize();
    if (errSync != cudaSuccess)
        printf("Sync kernel error: %s\n", cudaGetErrorString(errSync));
    if (errAsync != cudaSuccess)
        printf("Async kernel error: %s\n", cudaGetErrorString(errAsync));

    // Copy the result matrix back to host memory
    cudaMemcpy(c, f, row1 * col2 * sizeof(int), cudaMemcpyDeviceToHost);

    // Print the result
    printf("\nProduct of two matrices:\n");
    for (i = 0; i < row1; i++) {
        for (j = 0; j < col2; j++) {
            printf("%d\t", c[i][j]);
        }
        printf("\n");
    }

    // Free device memory
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}



Enter elements of first matrix of size 2x3

Enter elements of second matrix of size 3x2
Sync kernel error: the provided PTX was compiled with an unsupported toolchain.

Product of two matrices:
0	0	
0	0	



In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>

#define row1 2  // Number of rows of first matrix
#define col1 3  // Number of columns of first matrix
#define row2 3  // Number of rows of second matrix
#define col2 2  // Number of columns of second matrix

__global__ void matproduct(int *l, int *m, int *n) {
    int x = blockIdx.x;
    int y = blockIdx.y;
    int k;

    n[col2 * y + x] = 0;
    for (k = 0; k < col1; k++) {
        n[col2 * y + x] += l[col1 * y + k] * m[col2 * k + x];
    }
}

int main() {
    int a[row1][col1];
    int b[row2][col2];
    int c[row1][col2];
    int *d, *e, *f;
    int i, j;

    printf("\nEnter elements of first matrix of size 2x3:\n");
    for (i = 0; i < row1; i++) {
        for (j = 0; j < col1; j++) {
            scanf("%d", &a[i][j]);
        }
    }

    printf("\nEnter elements of second matrix of size 3x2:\n");
    for (i = 0; i < row2; i++) {
        for (j = 0; j < col2; j++) {
            scanf("%d", &b[i][j]);
        }
    }

    cudaMalloc((void **)&d, row1 * col1 * sizeof(int));
    cudaMalloc((void **)&e, row2 * col2 * sizeof(int));
    cudaMalloc((void **)&f, row1 * col2 * sizeof(int));

    cudaMemcpy(d, a, row1 * col1 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, row2 * col2 * sizeof(int), cudaMemcpyHostToDevice);

    dim3 grid(col2, row1);  // 2D grid: (columns of result, rows of result)
    matproduct<<<grid, 1>>>(d, e, f);

    cudaMemcpy(c, f, row1 * col2 * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\nProduct of two matrices:\n");
    for (i = 0; i < row1; i++) {
        for (j = 0; j < col2; j++) {
            printf("%d\t", c[i][j]);
        }
        printf("\n");
    }

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}



Enter elements of first matrix of size 2x3:

Enter elements of second matrix of size 3x2:

Product of two matrices:
0	0	
0	0	



In [ ]:
%%cuda
#include<stdio.h>
#include<cuda.h>

__global__ void arradd(int *x, int *y, int *z, int N) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < N) {  // Ensure the thread index is within bounds
        z[id] = x[id] + y[id];
    }
}

int main() {
    int N = 1 << 20;  // Size of the vectors (1 million elements)
    int size = N * sizeof(int);

    int *a, *b, *c;  // Host arrays
    int *d, *e, *f;  // Device arrays
    int i;

    // Allocate memory for the vectors on the host (CPU)
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize the vectors with some values (e.g., integers from 1 to N)
    for(i = 0; i < N; i++) {
        a[i] = i + 1;  // Vector A: 1, 2, 3, ..., N
        b[i] = i + 1;  // Vector B: 1, 2, 3, ..., N
    }

    // Allocate memory on the device (GPU)
    cudaMalloc((void**)&d, size);
    cudaMalloc((void**)&e, size);
    cudaMalloc((void**)&f, size);

    // Copy data from host to device
    cudaMemcpy(d, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, size, cudaMemcpyHostToDevice);

    // Set block size and grid size
    int blockSize = 256;  // Number of threads per block
    int gridSize = (N + blockSize - 1) / blockSize;  // Calculate number of blocks

    // Launch the kernel to add vectors
    arradd<<<gridSize, blockSize>>>(d, e, f, N);

    // Copy the result from device to host
    cudaMemcpy(c, f, size, cudaMemcpyDeviceToHost);

    // Print the first 10 elements of the result (for verification)
    printf("Sum of the first 10 elements of two large vectors:\n");
    for(i = 0; i < 10; i++) {
        printf("%d ", c[i]);
    }
    printf("\n");

    // Free allocated memory
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    free(a);
    free(b);
    free(c);

    return 0;
}


Sum of the first 10 elements of two large vectors:
0 0 0 0 0 0 0 0 0 0 

